## Subgrounds Tutorials: RocketPool_Snapshot

In this example, we are using  SyntheticFields to create new fields that do not exist in the original data source. These new fields are calculated based on existing fields in the financialsDailySnapshots entity.

First, we create a FieldPath object to query the required fields from the financialsDailySnapshots entity, sorted by timestamp in descending order and limited to the last 30 entries.

Next, we create a synthetic field `datetime` in the FinancialsDailySnapshot entity to convert the timestamp field into a human-readable format.

Then, we define three more synthetic fields. 

- `revDiffRevenueUSD30` calculates the difference in revenue between the dailyProtocolSideRevenueUSD and dailySupplySideRevenueUSD fields over the past 30 days. 

- `avgDailyRevenueUSD30` calculates the average of the dailyProtocolSideRevenueUSD and dailySupplySideRevenueUSD fields over the past 30 days.

- `revenueMargin30` calculates the percentage change in revenue between the dailyProtocolSideRevenueUSD and dailySupplySideRevenueUSD fields over the past 30 days.

Finally, we query the required fields, including the newly created synthetic fields, and flatten the data into a single DataFrame for analysis.

In [1]:
from datetime import datetime
from subgrounds.subgraph import SyntheticField
from subgrounds import Subgrounds
import pandas as pd

# Initialize Subgrounds
sg = Subgrounds()

# Load a subgraph using its API URL
financials_daily_snapshots = sg.load_subgraph('https://api.thegraph.com/subgraphs/name/messari/rocket-pool-ethereum')

# Create a FieldPath object for the required fields in the financialsDailySnapshots entity.
# Specify options to sort the data by timestamp in descending order and limit the number of results to 30.
financials_daily_snapshot_30days = financials_daily_snapshots.Query.financialsDailySnapshots(
    orderBy=financials_daily_snapshots.FinancialsDailySnapshot.timestamp, 
    orderDirection='desc', 
    first=30
)

# Create a SyntheticField on the Swap entity called `timestamp`, which will format the timestamp field into something more human readable
financials_daily_snapshots.FinancialsDailySnapshot.datetime = SyntheticField(
  lambda timestamp: str(datetime.fromtimestamp(timestamp)),
  SyntheticField.FLOAT,
  financials_daily_snapshots.FinancialsDailySnapshot.timestamp
)

# Define a synthetic field to calculate the daily revenue difference for the past 30 days
financials_daily_snapshots.FinancialsDailySnapshot.revDiffRevenueUSD30 = SyntheticField(
    lambda x, y: abs(x - y),
    SyntheticField.FLOAT,
    [financials_daily_snapshots.FinancialsDailySnapshot.dailyProtocolSideRevenueUSD,
    financials_daily_snapshots.FinancialsDailySnapshot.dailySupplySideRevenueUSD],
)

# the average daily revenue
financials_daily_snapshots.FinancialsDailySnapshot.avgDailyRevenueUSD30 = SyntheticField(
    lambda x, y: (x + y) / 2,
    SyntheticField.FLOAT,
    [financials_daily_snapshots.FinancialsDailySnapshot.dailyProtocolSideRevenueUSD,
    financials_daily_snapshots.FinancialsDailySnapshot.dailySupplySideRevenueUSD],
)

# Create a SyntheticField for the percentage change in daily revenue over the past 30 days
financials_daily_snapshots.FinancialsDailySnapshot.revenueMargin30  = SyntheticField(
    lambda x, y: x / y,
    SyntheticField.FLOAT,
    [financials_daily_snapshots.FinancialsDailySnapshot.dailyProtocolSideRevenueUSD,
    financials_daily_snapshots.FinancialsDailySnapshot.dailySupplySideRevenueUSD],
)


# Query data flattened to a single DataFrame
sg.query_df([
    financials_daily_snapshot_30days.timestamp,
    financials_daily_snapshot_30days.datetime,
    financials_daily_snapshot_30days.blockNumber,
    financials_daily_snapshot_30days.cumulativeProtocolSideRevenueUSD,
    financials_daily_snapshot_30days.cumulativeProtocolSideRevenueUSD,
    financials_daily_snapshot_30days.cumulativeSupplySideRevenueUSD,
    financials_daily_snapshot_30days.cumulativeTotalRevenueUSD,
    financials_daily_snapshot_30days.dailyProtocolSideRevenueUSD,
    financials_daily_snapshot_30days.dailySupplySideRevenueUSD,
    financials_daily_snapshot_30days.dailyTotalRevenueUSD,
    financials_daily_snapshot_30days.revDiffRevenueUSD30,
    financials_daily_snapshot_30days.avgDailyRevenueUSD30,
    financials_daily_snapshot_30days.revenueMargin30,

])

,financialsDailySnapshots_timestamp,financialsDailySnapshots_datetime,financialsDailySnapshots_blockNumber,financialsDailySnapshots_cumulativeProtocolSideRevenueUSD,financialsDailySnapshots_cumulativeSupplySideRevenueUSD,financialsDailySnapshots_cumulativeTotalRevenueUSD,financialsDailySnapshots_dailyProtocolSideRevenueUSD,financialsDailySnapshots_dailySupplySideRevenueUSD,financialsDailySnapshots_dailyTotalRevenueUSD,financialsDailySnapshots_revDiffRevenueUSD30,financialsDailySnapshots_avgDailyRevenueUSD30,financialsDailySnapshots_revenueMargin30
0,1678853147,2023-03-14 23:05:47,16830819,1.576503e+06,9.449043e+06,1.102555e+07,0.000000,16398.904951,16398.904951,16398.904951,8199.452475,0.000000
1,1678783139,2023-03-14 03:38:59,16825052,1.576503e+06,9.432644e+06,1.100915e+07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,1678643327,2023-03-12 12:48:47,16813532,1.576503e+06,9.432644e+06,1.100915e+07,4840.176758,28250.482666,33090.659424,23410.305908,16545.329712,0.171331
3,1678573343,2023-03-11 16:22:23,16807772,1.571663e+06,9.404394e+06,1.097606e+07,20543.551319,119905.794649,140449.345969,99362.243330,70224.672984,0.171331
4,1678433159,2023-03-10 01:25:59,16796251,1.551119e+06,9.284488e+06,1.083561e+07,4491.019712,26212.570504,30703.590216,21721.550792,15351.795108,0.171331
5,1678363043,2023-03-09 05:57:23,16790497,1.546628e+06,9.258275e+06,1.080490e+07,9030.886334,52710.243978,61741.130312,43679.357645,30870.565156,0.171331
6,1678223279,2023-03-07 15:07:59,16778991,1.537598e+06,9.205565e+06,1.074316e+07,9401.209628,54871.696408,64272.906036,45470.486780,32136.453018,0.171331
7,1678083023,2023-03-06 00:10:23,16767448,1.528196e+06,9.150693e+06,1.067889e+07,5252.416150,30656.585248,35909.001398,25404.169098,17954.500699,0.171331
8,1678013171,2023-03-05 04:46:11,16761693,1.522944e+06,9.120037e+06,1.064298e+07,9403.072566,54882.569751,64285.642317,45479.497184,32142.821158,0.171331
9,1677873035,2023-03-03 13:50:35,16750163,1.513541e+06,9.065154e+06,1.057870e+07,10590.856681,61815.265850,72406.122531,51224.409170,36203.061266,0.171331
